In [2]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from ape import accounts, chain, Contract, networks, project

In [ ]:
# SEE: https://gist.github.com/banteg/dcf6082ff7fc6ad51ce220146f29d9ff
networks.parse_network_choice('ethereum:mainnet:alchemy').__enter__()

In [ ]:
# mrgl v1 constants
MAINTENANCE_UNIT = int(1e6)
SECONDS_AGO = 43200  # 12 hr TWAP

In [ ]:
# UNI/WETH from Feb 27, 2023 to  26, 2024
FILEPATH = "results/MarginalV1LPRunner_0x1d42064Fc4Beb5F8aAF85F4617AE8b3b5B8Bd801_250000_0.25_0.25_3.5_0_50400_0.005_17998181_19311400_2400.csv"

In [ ]:
[
    _,
    pool_addr,
    maintenance,
    utilization,
    skew,
    leverage,
    rel_margin_above_safe_min,
    blocks_held,
    sqrt_price_tol,
    start,
    stop,
    step
] = re.split("_", FILEPATH[len("results/"):-len(".csv")])

In [ ]:
maintenance = int(maintenance)
utilization = float(utilization)
skew = float(skew)
leverage = float(leverage)
rel_margin_above_safe_min = float(rel_margin_above_safe_min)
blocks_held = int(blocks_held)
sqrt_price_tol = float(sqrt_price_tol)
start = int(start)
stop = int(stop)
step = int(step)

In [ ]:
leverage

In [ ]:
# values array from backtester for marginal LP pool
# values = [liquidity, sqrtPriceX96, liquidityLocked, lpTotalLiquidity, lpAmount0, lpAmount1]
df = pd.read_csv(FILEPATH)
df

In [ ]:
df.columns

In [ ]:
# plot mrgl pool tick vs oracle time weighted average tick over time
def calc_tick_from_sqrt_price_x96(sqrt_price_x96: int) -> int:
    price = (int(sqrt_price_x96) ** 2) / (1 << 192)
    return int(np.log(price) / np.log(1.0001))

In [ ]:
df['tick'] = df['values1'].apply(calc_tick_from_sqrt_price_x96)
df['tick']

In [ ]:
def calc_time_weighted_average_tick(tick_cumulatives_diff: int) -> int:
    return int(tick_cumulatives_diff) // int(SECONDS_AGO)

In [ ]:
# observation0 at timestamp - seconds_ago; observation1 at timestamp
df['oracle_tick'] = (df['univ3_observation1_tick_cumulatives'] - df['univ3_observation0_tick_cumulatives']).apply(calc_time_weighted_average_tick)
df['oracle_tick']

In [ ]:
df.plot(x='number', y=['tick', 'oracle_tick'])

In [ ]:
# zoom in to a 5 day window at end to see oracle tick timelag
# 12 hr TWAP lags so insurance mechanism for LPs must be robust to guarantee no bad debt
df[df['timestamp'] > df['timestamp'].iloc[-1] - 5 * 86400].plot(x='number', y=['tick', 'oracle_tick'])

In [ ]:
# plot mrgl pool price vs oracle time weighted average price over time
def calc_price_from_tick(tick: int) -> float:
    return 1.0001 ** tick

In [ ]:
df['price'] = df['tick'].apply(calc_price_from_tick)
df['price']

In [ ]:
df['oracle_price'] = df['oracle_tick'].apply(calc_price_from_tick)
df['oracle_price']

In [ ]:
fp = "assets/" + FILEPATH[len("results/"):-len(".csv")] + "-prices.png"
ax = df.plot(x='number', y=['price', 'oracle_price'])
ax.get_figure().savefig(fp)

In [ ]:
df[df['timestamp'] > df['timestamp'].iloc[-1] - 5 * 86400].plot(x='number', y=['price', 'oracle_price'])

In [ ]:
# plot liquidity gained over time. convert to datatype first
df['lp_total_liquidity'] = df['values3'].apply(lambda v: int(v))
df['lp_amount0'] = df['values4'].apply(lambda v: int(v))
df['lp_amount1'] = df['values5'].apply(lambda v: int(v))

In [ ]:
df

In [ ]:
df['net_liquidity_settled_cumulative_zero_for_one'] = df['_net_liquidity_settled_cumulative0'].apply(lambda v: int(v))
df['net_liquidity_settled_cumulative_one_for_zero'] = df['_net_liquidity_settled_cumulative1'].apply(lambda v: int(v))
df['net_liquidity_settled_cumulative'] = df['net_liquidity_settled_cumulative_zero_for_one'] + df['net_liquidity_settled_cumulative_one_for_zero']

df['net_liquidity_liquidated_cumulative_zero_for_one'] = df['_net_liquidity_liquidated_cumulative0'].apply(lambda v: int(v))
df['net_liquidity_liquidated_cumulative_one_for_zero'] = df['_net_liquidity_liquidated_cumulative1'].apply(lambda v: int(v))
df['net_liquidity_liquidated_cumulative'] = df['net_liquidity_liquidated_cumulative_zero_for_one'] + df['net_liquidity_liquidated_cumulative_one_for_zero']

df['net_liquidity_swap_fees_cumulative'] = df['_net_liquidity_swap_fees_cumulative'].apply(lambda v: int(v))
df['net_liquidity_position_fees_cumulative'] = df['_net_liquidity_position_fees_cumulative'].apply(lambda v: int(v))
df['net_liquidity_fees_cumulative'] = df['net_liquidity_swap_fees_cumulative'] + df['net_liquidity_position_fees_cumulative']

In [ ]:
df['net_liquidity_total_cumulative'] = df['lp_total_liquidity'] - df['lp_total_liquidity'].iloc[0]

In [ ]:
df

In [ ]:
# put in yield terms relative to initial liquidity
df['lp_total_liquidity'].iloc[0]

In [ ]:
df['yield_liquidity_total'] = (df['lp_total_liquidity'] / df['lp_total_liquidity'].iloc[0] - 1)
df['yield_liquidity_fees'] = (df['net_liquidity_fees_cumulative'] / df['lp_total_liquidity'].iloc[0])
df['yield_liquidity_settled'] = (df['net_liquidity_settled_cumulative'] / df['lp_total_liquidity'].iloc[0])
df['yield_liquidity_liquidated'] = (df['net_liquidity_liquidated_cumulative'] / df['lp_total_liquidity'].iloc[0])

In [ ]:
df

In [ ]:
fp = "assets/" + FILEPATH[len("results/"):-len(".csv")] + "-yields.png"
ax = df.plot(x='number', y=['yield_liquidity_total', 'yield_liquidity_fees', 'yield_liquidity_settled', 'yield_liquidity_liquidated'])
ax.get_figure().savefig(fp)

In [ ]:
# look at net liquidity liquidated over time and check no shortfalls due to liquidations (i.e. no bad debt)
fp = "assets/" + FILEPATH[len("results/"):-len(".csv")] + "-yields-liquidated.png"
ax = df.plot(x='number', y=['yield_liquidity_liquidated'], color=['r'])
ax.get_figure().savefig(fp)

In [ ]:
# focus on last 5 day window ...
fp = "assets/" + FILEPATH[len("results/"):-len(".csv")] + "-price_vs_liquidations_yield_T-5.png"
fig, ax = plt.subplots(2, 1)
df[df['timestamp'] > df['timestamp'].iloc[-1] - 5 * 86400].plot(x='number', y=['price', 'oracle_price'], ax=ax[0])
df[df['timestamp'] > df['timestamp'].iloc[-1] - 5 * 86400].plot(x='number', y=['yield_liquidity_liquidated'], color=['r'], ax=ax[1])
fig.savefig(fp)

In [ ]:
# and for settled? dependent on funding rates. order of mag smaller than contribution from liquidations
fp = "assets/" + FILEPATH[len("results/"):-len(".csv")] + "-yields-settled.png"
ax = df.plot(x='number', y=['yield_liquidity_settled'], color=['g'])
ax.get_figure().savefig(fp)

In [ ]:
def calc_reserves_from_liquidity_sqrt_price_x96(sqrt_price_x96: int, liquidity: int) -> (int, int):
    amount0 = (int(liquidity) << 96) // int(sqrt_price_x96)
    amount1 = (int(liquidity) * int(sqrt_price_x96)) // (1 << 96)
    return (amount0, amount1)

In [ ]:
# check actual token balances in the pool can cover pool liquidity post-settle, liquidate of outstanding positions
df['balance0_pool'] = df['_balances_pool0'].apply(lambda v: int(v))
df['balance1_pool'] = df['_balances_pool1'].apply(lambda v: int(v))

In [ ]:
df["liquidity"] = df["values0"].apply(lambda v: int(v))
df["sqrtPriceX96"] = df["values1"].apply(lambda v: int(v))

In [ ]:
df['reserve0'] = df.apply(lambda x: calc_reserves_from_liquidity_sqrt_price_x96(x.sqrtPriceX96, x.liquidity)[0], axis=1)
df['reserve1'] = df.apply(lambda x: calc_reserves_from_liquidity_sqrt_price_x96(x.sqrtPriceX96, x.liquidity)[1], axis=1)

In [ ]:
df['amounts0_locked_zero_for_one'] = df['_amounts0_locked0'].apply(lambda v: int(v))
df['amounts1_locked_zero_for_one'] = df['_amounts1_locked0'].apply(lambda v: int(v))
df['amounts0_locked_one_for_zero'] = df['_amounts0_locked1'].apply(lambda v: int(v))
df['amounts1_locked_one_for_zero'] = df['_amounts1_locked1'].apply(lambda v: int(v))

In [ ]:
df['collaterals_zero_for_one'] = df['_sizes_outstanding0'].apply(lambda v: int(v)) + df['_margins_outstanding0'].apply(lambda v: int(v))
df['debts_zero_for_one'] = df['_debts_outstanding0'].apply(lambda v: int(v))
df['collaterals_one_for_zero'] = df['_sizes_outstanding1'].apply(lambda v: int(v)) + df['_margins_outstanding1'].apply(lambda v: int(v))
df['debts_one_for_zero'] = df['_debts_outstanding1'].apply(lambda v: int(v))

In [ ]:
df['reserve0_liquidated'] = df['reserve0'] + df['amounts0_locked_zero_for_one'] + df['amounts0_locked_one_for_zero']
df['reserve1_liquidated'] = df['reserve1'] + df['amounts1_locked_zero_for_one'] + df['amounts1_locked_one_for_zero']

In [ ]:
df.filter(['reserve0_liquidated', 'balance0_pool', 'reserve1_liquidated', 'balance1_pool'])

In [ ]:
# should be some dust left over in pool or very little negative amounts (won't be stuck due to transfer helper lib)
df['net_balance0_pool'] = df['balance0_pool'] - df['reserve0_liquidated']
df['net_balance1_pool'] = df['balance1_pool'] - df['reserve1_liquidated']

In [ ]:
df['rel_balance0_pool'] = df['net_balance0_pool'] / df['reserve0_liquidated']
df['rel_balance1_pool'] = df['net_balance1_pool'] / df['reserve1_liquidated']

In [ ]:
# dust for balance check
df.filter(['net_balance0_pool', 'net_balance1_pool', 'rel_balance0_pool', 'rel_balance1_pool'])

In [ ]:
print(f"last excess reserves (relative): {(df['rel_balance0_pool'].iloc[-1], df['rel_balance1_pool'].iloc[-1])}")

In [ ]:
df.plot(x='number', y=['_positions_liquidated_cumulative0', '_positions_liquidated_cumulative1'])

In [ ]:
df['sizes_outstanding_zero_for_one'] = df['_sizes_outstanding0'].apply(lambda v: int(v))
df['sizes_outstanding_one_for_zero'] = df['_sizes_outstanding1'].apply(lambda v: int(v))

In [ ]:
# relative to initial total liquidity x, y amounts
df['rel_sizes_outstanding_zero_for_one'] = df['sizes_outstanding_zero_for_one'] / df['lp_amount1'].iloc[0]  # size in token1
df['rel_sizes_outstanding_one_for_zero'] = df['sizes_outstanding_one_for_zero'] / df['lp_amount0'].iloc[0]  # size in token0

In [ ]:
fig, ax = plt.subplots(2, 1)
df[1:].plot(x='number', y=['rel_sizes_outstanding_zero_for_one'], ax=ax[0], color=['C0'])
df[1:].plot(x='number', y=['rel_sizes_outstanding_one_for_zero'], ax=ax[1], color=['C1'])

In [ ]:
df['sizes_liquidated_cumulative_zero_for_one'] = df['_sizes_liquidated_cumulative0'].apply(lambda v: int(v))
df['sizes_liquidated_cumulative_one_for_zero'] = df['_sizes_liquidated_cumulative1'].apply(lambda v: int(v))
df['sizes_settled_cumulative_zero_for_one'] = df['_sizes_settled_cumulative0'].apply(lambda v: int(v))
df['sizes_settled_cumulative_one_for_zero'] = df['_sizes_settled_cumulative1'].apply(lambda v: int(v))

In [ ]:
# relative to initial total liquidity x, y amounts
df['rel_sizes_liquidated_cumulative_zero_for_one'] = df['sizes_liquidated_cumulative_zero_for_one'] / df['lp_amount1'].iloc[0]  # size in token1
df['rel_sizes_liquidated_cumulative_one_for_zero'] = df['sizes_liquidated_cumulative_one_for_zero'] / df['lp_amount0'].iloc[0]  # size in token0
df['rel_sizes_settled_cumulative_zero_for_one'] = df['sizes_settled_cumulative_zero_for_one'] / df['lp_amount1'].iloc[0]  # size in token1
df['rel_sizes_settled_cumulative_one_for_zero'] = df['sizes_settled_cumulative_one_for_zero'] / df['lp_amount0'].iloc[0]  # size in token0

In [ ]:
fp = "assets/" + FILEPATH[len("results/"):-len(".csv")] + "-rel-sizes-liquidated-settled.png"
fig, ax = plt.subplots(4, 1)
df.plot(x='number', y=['rel_sizes_liquidated_cumulative_zero_for_one'], ax=ax[0], color='C0')
df.plot(x='number', y=['rel_sizes_liquidated_cumulative_one_for_zero'], ax=ax[1], color='C1')
df.plot(x='number', y=['rel_sizes_settled_cumulative_zero_for_one'], ax=ax[2], color='C2')
df.plot(x='number', y=['rel_sizes_settled_cumulative_one_for_zero'], ax=ax[3], color='C3')
fig.savefig(fp)

In [ ]:
# examine yields in token0 or token1 terms ...
df['lp_value0_total'] = (2 * df['lp_total_liquidity'] * (1 << 96)) // df['sqrtPriceX96']
df['lp_value1_total'] = (2 * df['lp_total_liquidity'] * df['sqrtPriceX96']) // (1 << 96)

In [ ]:
df['yield_value0_total'] = df['lp_value0_total'] / df['lp_value0_total'].iloc[0] - 1
df['yield_value1_total'] = df['lp_value1_total'] / df['lp_value1_total'].iloc[0] - 1

In [ ]:
df.filter(['lp_value0_total', 'lp_value1_total', 'yield_value0_total', 'yield_value1_total'])

In [ ]:
# yield in value terms ...
fp = "assets/" + FILEPATH[len("results/"):-len(".csv")] + "-yields-value.png"
fig, ax = plt.subplots(2, 1)
df.plot(x='number', y=['yield_value0_total'], ax=ax[0], color='C0')
df.plot(x='number', y=['yield_value1_total'], ax=ax[1], color='C1')
fig.savefig(fp)

In [ ]:
# benchmark against passive hold ...
df['passive_value0_total'] = df['lp_amount0'].iloc[0] + df['lp_amount1'].iloc[0] / df['price']
df['passive_value1_total'] = df['lp_amount0'].iloc[0] * df['price'] + df['lp_amount1'].iloc[0]

In [ ]:
df['yield_value0_total_lp_to_passive'] = df['lp_value0_total'] / df['passive_value0_total'] - 1
df['yield_value1_total_lp_to_passive'] = df['lp_value1_total'] / df['passive_value1_total'] - 1

df['yield_value0_total_lp_to_passive'] = df['lp_value0_total'] / df['passive_value0_total'] - 1
df['yield_value1_total_lp_to_passive'] = df['lp_value1_total'] / df['passive_value1_total'] - 1

In [ ]:
df.filter(['passive_value0_total', 'passive_value1_total', 'yield_value0_total_lp_to_passive', 'yield_value1_total_lp_to_passive'])

In [ ]:
# similar yield increases to liquidity terms but with differences due to IL
fp = "assets/" + FILEPATH[len("results/"):-len(".csv")] + "-yields-value-rel-to-passive.png"
fig, ax = plt.subplots(2, 1)
df.plot(x='number', y=['yield_value0_total_lp_to_passive'], ax=ax[0], color='C0')
df.plot(x='number', y=['yield_value1_total_lp_to_passive'], ax=ax[1], color='C1')
fig.savefig(fp)

In [ ]:
# yield loss to impermanent loss
def calc_impermanent_loss(sqrt_ratio: float) -> float:
    return (2 * sqrt_ratio) / (sqrt_ratio **2 + 1) - 1

In [ ]:
sqrt_ratio_end = df['sqrtPriceX96'].iloc[-1] / df['sqrtPriceX96'].iloc[0]
calc_impermanent_loss(sqrt_ratio_end)

In [ ]:
# plot IL (approx)
df['yield_il'] = df['yield_value1_total_lp_to_passive'] - df['yield_liquidity_total']

In [ ]:
fp = "assets/" + FILEPATH[len("results/"):-len(".csv")] + "-yields-il.png"
ax = df.plot(x='number', y=['yield_il'], color='C3')
ax.get_figure().savefig(fp)

In [ ]:
# check out total volumes on both sides (count based at settlement/liquidation given columns)
df['rel_volumes_closed_cumulative_zero_for_one'] = df['rel_sizes_liquidated_cumulative_zero_for_one'] + df['rel_sizes_settled_cumulative_zero_for_one']
df['rel_volumes_closed_cumulative_one_for_zero'] = df['rel_sizes_liquidated_cumulative_one_for_zero'] + df['rel_sizes_settled_cumulative_one_for_zero']

In [ ]:
df.filter(['rel_volumes_closed_cumulative_zero_for_one', 'rel_volumes_closed_cumulative_one_for_zero'])

In [ ]:
# cumulative volumes relative to initial TVL
fp = "assets/" + FILEPATH[len("results/"):-len(".csv")] + "-rel_volumes_cumulative.png"
fig, ax = plt.subplots(2, 1)
df.plot(x='number', y=['rel_volumes_closed_cumulative_zero_for_one'], ax=ax[0], color='C0')
df.plot(x='number', y=['rel_volumes_closed_cumulative_one_for_zero'], ax=ax[1], color='C1')
fig.savefig(fp)

In [ ]:
# zoom in over last blocks held period for sim to show turn over
fp = "assets/" + FILEPATH[len("results/"):-len(".csv")] + "-rel_volumes_cumulative_T-blocks_held.png"
fig, ax = plt.subplots(2, 1)
df[df['timestamp'] > df['timestamp'].iloc[-1] - 12 * blocks_held].plot(x='number', y=['rel_volumes_closed_cumulative_zero_for_one'], ax=ax[0], color='C0')
df[df['timestamp'] > df['timestamp'].iloc[-1] - 12 * blocks_held].plot(x='number', y=['rel_volumes_closed_cumulative_one_for_zero'], ax=ax[1], color='C1')
fig.savefig(fp)